In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from torch.utils.data import DataLoader
from torchvision import models,transforms  # Add this import at the top
from tqdm import tqdm  # For progress bar
import os
from PIL import Image


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Define a custom ResNet model with added dropout and a fully connected layer
class CustomResNet(nn.Module):
    def __init__(self, num_classes):
        super(CustomResNet, self).__init__()
        self.model = models.resnet101(weights=models.ResNet101_Weights.IMAGENET1K_V2)
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        return self.model(x)

In [4]:
# Define dataset class
class LungImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_folder, classes, transform=None):
        self.dataset_folder = dataset_folder
        self.transform = transform
        self.classes = classes
        self.data = self._load_data()

    def _load_data(self):
        data = []
        label_mapping = {class_name: idx for idx, class_name in enumerate(self.classes)}

        for class_name in self.classes:
            class_folder = os.path.join(self.dataset_folder, class_name)
            if os.path.isdir(class_folder):
                for img_name in os.listdir(class_folder):
                    img_path = os.path.join(class_folder, img_name)
                    if os.path.isfile(img_path):
                        data.append((img_path, label_mapping[class_name]))

        print(f"Loaded {len(data)} images from {self.dataset_folder}")
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

In [5]:
def test(model, test_loader, device):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []

    with torch.no_grad():  # Disable gradient calculation for testing
        for images, labels in tqdm(test_loader, desc="Testing"):
            images, labels = images.to(device), labels.to(device)

            # Get model predictions
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            # Collect labels and predictions
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    # Calculate accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")

    # Optional: Classification report (precision, recall, f1-score)
    print("Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=test_loader.dataset.classes))

    return accuracy

def main():
    # Same setup as training, but load the best model and test data
    dataset_folder = "C:\\Users\\Acer\\Desktop\\Model\\Split"
    classes = ['Atelectasis', 'Covid19', 'Emphysema', 'Consolidation', 'Cardiomegaly', 'Infiltration', 'Nodule', 'Pneumonia', 'NORMAL', 'Pneumothorax']

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    test_dataset = LungImageDataset(dataset_folder=os.path.join(dataset_folder, "test"), classes=classes, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Load the best model
    model = CustomResNet(num_classes=len(classes)).to(device)
    model.load_state_dict(torch.load("newmodel.pth"))

    # Test the model
    test_accuracy = test(model, test_loader, device)
    print(f"Final Test Accuracy: {test_accuracy * 100:.2f}%")

if __name__ == "__main__":
    main()


Loaded 7408 images from C:\Users\Acer\Desktop\Model\Split\test


C:\Users\Acer\AppData\Local\Temp\ipykernel_16048\3774374154.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("newmodel.pth"))
Testing: 1

Test Accuracy: 93.39%
Classification Report:
               precision    recall  f1-score   support

  Atelectasis       0.80      0.86      0.83       758
      Covid19       0.99      1.00      0.99       669
    Emphysema       0.98      0.99      0.99       747
Consolidation       0.96      0.99      0.98       763
 Cardiomegaly       0.96      0.99      0.98       777
 Infiltration       0.78      0.72      0.75       748
       Nodule       0.91      0.86      0.89       746
    Pneumonia       1.00      0.99      0.99       742
       NORMAL       0.99      1.00      1.00       713
 Pneumothorax       0.96      0.94      0.95       745

     accuracy                           0.93      7408
    macro avg       0.93      0.93      0.93      7408
 weighted avg       0.93      0.93      0.93      7408

Final Test Accuracy: 93.39%
